In [1]:
import pandas as pd
import plotly.express as px

C:\Users\johan\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def count_nan(column):
    data[column].isnull().sum()

In [13]:
data = pd.read_csv('./Data/EU_LNG_data.csv')

"""
LNG - DTMI declared total maximum inventory
dtrs maximum send out
inventory 10^3 m^3 LNG
sendOut GWh/d
DTMI 10^3 m^3 LNG
DTRS GWh/d
"""

'\nLNG - DTMI declared total maximum inventory\ndtrs maximum send out\ninventory 10^3 m^3 LNG\nsendOut GWh/d\nDTMI 10^3 m^3 LNG\nDTRS GWh/d\n'

In [11]:
data.head()

,Unnamed: 0,name,code,url,gasDayStart,inventory,sendOut,dtmi,dtrs,info,status
0,0,Belgium,BE,BE,2022-12-02,539.78,268.6,566.0,541.0,[],C
1,1,Belgium,BE,BE,2022-12-01,558.22,339.5,566.0,541.0,[],C
2,2,Belgium,BE,BE,2022-11-30,342.31,466.9,566.0,541.0,[],C
3,3,Belgium,BE,BE,2022-11-29,206.58,487.3,566.0,541.0,[],C
4,4,Belgium,BE,BE,2022-11-28,408.93,509.6,566.0,541.0,[],C


In [14]:
data.status.unique()
# column status not relevant only concists of c and e

array(['C', 'E'], dtype=object)

In [15]:
print(data['info'].unique())
# column info also not relevant ['[]' "['/news/52124']"]

['[]' "['/news/52124']"]


In [16]:
print(data.dtypes)
# convert gasDayStart to dtype datetime

Unnamed: 0       int64
name            object
code            object
url             object
gasDayStart     object
inventory      float64
sendOut        float64
dtmi           float64
dtrs           float64
info            object
status          object
dtype: object


In [17]:
data['gasDayStart'] = pd.to_datetime(data['gasDayStart'])
print(data.inventory.value_counts())
# checking for nans

133.17     66
86.80       6
105.75      6
238.28      6
118.10      4
           ..
84.99       2
112.20      2
127.01      2
151.02      2
1314.82     2
Name: inventory, Length: 2858, dtype: int64


In [18]:
for i in data.columns:
    print(count_nan(i))
# no nans

None
None
None
None
None
None
None
None
None
None
None


In [19]:
# convert inventory and dtmi to GWh
data[['inventory', 'dtmi']] = data[['inventory', 'dtmi']].apply(lambda x: (((x * 1000)*11.6)/1000000))
print(data)

      Unnamed: 0     name code url gasDayStart  inventory  sendOut     dtmi  \
0              0  Belgium   BE  BE  2022-12-02   6.261448    268.6   6.5656   
1              1  Belgium   BE  BE  2022-12-01   6.475352    339.5   6.5656   
2              2  Belgium   BE  BE  2022-11-30   3.970796    466.9   6.5656   
3              3  Belgium   BE  BE  2022-11-29   2.396328    487.3   6.5656   
4              4  Belgium   BE  BE  2022-11-28   4.743588    509.6   6.5656   
...          ...      ...  ...  ..         ...        ...      ...      ...   
5995        5995    Spain   ES  ES  2022-02-10  13.380252    991.7  38.4714   
5996        5996    Spain   ES  ES  2022-02-09  15.148788    950.3  38.4714   
5997        5997    Spain   ES  ES  2022-02-08  15.410600    983.2  38.4714   
5998        5998    Spain   ES  ES  2022-02-07  15.580076    842.3  38.4714   
5999        5999    Spain   ES  ES  2022-02-06  15.251912    653.0  38.4714   

        dtrs info status  
0      541.0   []      C

In [20]:
# creating subset
rel = data[['name', 'gasDayStart', 'inventory', 'sendOut', 'dtmi', 'dtrs']]
print(rel)
print(rel.dtypes)

         name gasDayStart  inventory  sendOut     dtmi    dtrs
0     Belgium  2022-12-02   6.261448    268.6   6.5656   541.0
1     Belgium  2022-12-01   6.475352    339.5   6.5656   541.0
2     Belgium  2022-11-30   3.970796    466.9   6.5656   541.0
3     Belgium  2022-11-29   2.396328    487.3   6.5656   541.0
4     Belgium  2022-11-28   4.743588    509.6   6.5656   541.0
...       ...         ...        ...      ...      ...     ...
5995    Spain  2022-02-10  13.380252    991.7  38.4714  1910.4
5996    Spain  2022-02-09  15.148788    950.3  38.4714  1910.4
5997    Spain  2022-02-08  15.410600    983.2  38.4714  1910.4
5998    Spain  2022-02-07  15.580076    842.3  38.4714  1910.4
5999    Spain  2022-02-06  15.251912    653.0  38.4714  1910.4

[6000 rows x 6 columns]
name                   object
gasDayStart    datetime64[ns]
inventory             float64
sendOut               float64
dtmi                  float64
dtrs                  float64
dtype: object


In [22]:
# basic analyse inventory and sendOut
basic = rel[['name', 'inventory', 'sendOut']].groupby(['name']).describe()
basic.to_csv('./Data/basic_ex.csv')


In [26]:
df = rel.round(decimals=2)
fig = px.box(df, x="name", y="inventory", )
fig.update_layout(
    xaxis_title='Country',
    yaxis_title='Inventory in GWh',
    title_text="Boxplot of EU Countries Inventories")
fig.show()

In [27]:
fig2 = px.box(df, x="name", y="sendOut")
fig2.update_layout(
    xaxis_title='Country',
    yaxis_title='send out in GWh/d',
    title_text="Boxplot of EU Countries Send-out Rate")
fig2.show()

In [28]:
data['percentage Inventory to DTMI'] = data['inventory']/data['dtmi']
data['percentage Inventory to DTMI'] = data['percentage Inventory to DTMI']*100

data['percentage sendOut of DTRS'] = data['sendOut']/data['dtrs']
data['percentage sendOut of DTRS'] = data['percentage sendOut of DTRS']*100

In [30]:
data = data.loc[::-1]
data.to_csv('./Data/reverse_data.csv')
countries = data['name'].unique()

for i in countries:
    selected_rows = data[data['name'] == i].round(decimals=2).sort_values("gasDayStart")

    fig = px.line(selected_rows, x='gasDayStart', y='percentage Inventory to DTMI',

                  title=f"Line Plot of percentage of Inventory to DTMI for {i}")
    fig.update_layout(xaxis_title='Dates', yaxis_title='percentage of total Inventroy')
    fig.show()

In [34]:
data.groupby('gasDayStart')['percentage sendOut of DTRS'].mean()

gasDayStart
2022-02-06    57.197294
2022-02-07    60.345894
2022-02-08    63.100196
2022-02-09    64.969809
2022-02-10    67.028438
                ...    
2022-11-28    78.411468
2022-11-29    79.313813
2022-11-30    79.827304
2022-12-01    70.716945
2022-12-02    71.247470
Name: percentage sendOut of DTRS, Length: 300, dtype: float64